In [15]:
%matplotlib inline
from __future__ import division;
from __future__ import print_function;
from __future__ import absolute_import;
import sys, os;
from collections import OrderedDict, namedtuple;
import numpy as np;

#import deepLIFT stuff
scriptsDir = os.environ.get("DEEPLIFT_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable DEEPLIFT_DIR to point to the deeplift code (WITHIN the deeplift repo)");
sys.path.insert(0,scriptsDir);
#Make sure the directory is set to import the lab's version of keras
scriptsDir = os.environ.get("KERAS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable KERAS_DIR");
sys.path.insert(0,scriptsDir)

import keras_conversion as kc
import theano
import models
from blobs import MxtsMode
import deeplift_util

In [2]:
reload(kc)
model_weights  = "mar31_diff_and_present_peaks_datasetLeuk_75M_logpval5_diffmethoddeseq_top400000_peaksize1000_jitter50_noGCmatch_softmaxDiffTasks_CustomModel_GraphSoftmax_currentBest_epoch_5_modelWeights.h5"
model_yaml = "mar31_diff_and_present_peaks_datasetLeuk_75M_logpval5_diffmethoddeseq_top400000_peaksize1000_jitter50_noGCmatch_softmaxDiffTasks_CustomModel_GraphSoftmax_currentBest_epoch_5_modelYaml.yaml"
keras_model = kc.load_keras_model(model_weights, model_yaml,
                                  normalise_conv_for_one_hot_encoded_input=True,
                                  name_of_conv_layer_to_normalise="conv1")

Using Theano backend.


In [ ]:
keras_model.inputs['input'].get_config()

In [ ]:
keras_model.nodes['softmax_for_col_LMPPvLSC_dense'].get_config()

In [ ]:
keras_model.nodes.keys()

In [ ]:
import keras_conversion as kc
import models
reload(models)
reload(kc)
import blobs
reload(blobs)
from blobs import MxtsMode
deeplift_model = kc.convert_graph_model(keras_model, mxts_mode=MxtsMode.DeepLIFT)

In [17]:
#let's forward prop a bunch of random sequences through it
random_sequences = np.random.uniform(size=(2000,1,4,1000))

In [ ]:
#will randomly check a couple of the outputs
outputs_to_check = ['softmax_for_col_HSCvBcell_activ', 'softmax_for_col_HSCvErythroid_activ']

In [ ]:
import theano
#compile theano functions to compute outputs for keras model
keras_theano_funcs = [
    theano.function([keras_model.inputs['input'].input],
                    keras_model.nodes[output_layer_name].get_output(),
                    allow_input_downcast=True)
    for output_layer_name in outputs_to_check
]

In [ ]:
#compile theano funcs to compute otuputs for deeplift model
deeplift_theano_funcs = [
    theano.function([deeplift_model.get_name_to_blob()['input'].get_activation_vars()],
                    deeplift_model.get_name_to_blob()[output_layer_name].get_activation_vars(),
                    allow_input_downcast=True) for output_layer_name in outputs_to_check
]

In [ ]:
#get the fprop results for both keras and deeplift
import deeplift_util
#run the functions on all the inputs
keras_results, deeplift_results = [[deeplift_util.run_function_in_batches(
                                     func, input_data_list=[random_sequences],
                                     batch_size=10, progress_update=None) for func in funcs]
                                    for funcs in [keras_theano_funcs, deeplift_theano_funcs]]

In [ ]:
#verify that they differ by tiny amounts
for keras_result, deeplift_result in zip(keras_results, deeplift_results):
    print("for single seq",np.sum(np.abs(np.array(keras_result[0])-np.array(deeplift_result[0]))))
    print("across all seq",np.sum(np.abs(np.array(keras_result)-np.array(deeplift_result))))

In [5]:
#make sure multiplier computation and stuff works
output_layer_name = 'softmax_for_col_HSCvBcell_dense'

In [ ]:
contribs_func = deeplift_model.get_target_contribs_func(
                 input_layer_name='input',
                 pre_activation_target_layer_name=output_layer_name)

In [ ]:
contribs = contribs_func(task_idx=2, input_data_list=[random_sequences], batch_size=10, progress_update=200)

In [6]:
#for checking purposes, will verify that gradient computation is what it should be
keras_gradient_function = theano.function([keras_model.inputs['input'].input],
                           theano.tensor.grad(theano.tensor.sum(keras_model.nodes[output_layer_name].get_output()[:,2]),
                                              keras_model.inputs['input'].input),
                           allow_input_downcast=True)

In [10]:
reload(kc)
reload(models)
gradient_model = kc.convert_graph_model(keras_model, mxts_mode=MxtsMode.Gradient)
gradients_func = gradient_model.get_target_multipliers_func(
                     input_layer_name='input',
                     pre_activation_target_layer_name=output_layer_name)

In [11]:
gradient_model_direct_gradient_function = theano.function([gradient_model.get_name_to_blob()['input'].get_activation_vars()],
                           theano.tensor.grad(theano.tensor.sum(gradient_model.get_name_to_blob()[output_layer_name].get_activation_vars()[:,2]),
                                              gradient_model.get_name_to_blob()['input'].get_activation_vars()),
                           allow_input_downcast=True)

In [21]:
gradients_deeplift = gradients_func(task_idx=2, input_data_list=[random_sequences[:10]], batch_size=1, progress_update=None)

In [19]:
gradients_keras = deeplift_util.run_function_in_batches(
                                     keras_gradient_function, input_data_list=[random_sequences[:10]], batch_size=1, progress_update=None)

In [18]:
gradients_direct = deeplift_util.run_function_in_batches(gradient_model_direct_gradient_function,
                                                         input_data_list=[random_sequences[:10]], batch_size=1, progress_update=None)

In [20]:
np.sum(np.abs(np.array(gradients_direct)-np.array(gradients_keras)))

72.753975

In [22]:
#uh oh
np.sum(np.abs(np.array(gradients_deeplift)-np.array(gradients_keras)))

93354.914

In [ ]:
gradients_deeplift

In [ ]:
gradients_keras